# DATA605 Final Project                                                                      
### Ling Cai


## About This File: 

###### Two different tools were used in this study.  Python Jupyter Notebook was used to cleaning and setting up the dataframe, with some visualization. Tableau was used for visualization only.  In this file, the Tableau images are embedded in the file. 

## About The Project: 

##### To this date, some students are still under the mind set that academic test score means everything. The purpose of this study is to drill down to a few variables, to determine their effectiveness and their impacts. 

In [106]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [107]:
import numpy as np
import scipy as sp
import scipy.spatial.distance as spd
import scipy.cluster.hierarchy as sph
import sklearn as sk
import sklearn.metrics as skm
import sklearn.cluster as skc
import sklearn.decomposition as skd
import sklearn.mixture as skmix
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='white', color_codes=True, font_scale=1.3)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

## 1. Set up the Data Frame

There are two data files used in this analysis. Both files are published by IPEDS (The Integrated Postsecondary Education Data System) in the U.S. The first files contains 2281 rows of data, each row represents a college. There are many variables, i.e. number of applicant, number of admission, number of enrollment, Calculated SAT CR Mean, Calculate SAT M Mean, etc. Some variables are not suited for the purpose of the study, therefore not used. The second file contains the graduation rate only. Both files contains some missing data of the useful variables. Data cleaning was done to 1. delete the un-necessary variables, 2. join the two tables, 3. delete the rows with null values in needed variable columns. 

In [108]:
# Import the first file. This is the main file. 

xls_file1 = pd.ExcelFile('ipeds original.xlsx')
df_ipeds_original = xls_file1.parse('Smaller file')
# df_ipeds_original = pd.read_excel('ipeds.xlsx', sheetname = 'Smaller file') # Alternatively you can use this line.
# df_ipeds_original.head(1) 

# Drop some un-necessary columns
df_ipeds_temp = df_ipeds_original.drop([ 'year','Calculated SAT CR Mean', 'Calculated SAT M Mean', 'Estimated enrollment, total', 'Estimated undergraduate enrollment, total', 'Estimated first-time degree/certificate-seeking undergraduate enrollment, total', 'Estimated graduate enrollment, total', 'Admissions yield  (percent of admitted freshmen enrolling)'
], axis=1)

# The regular .head() code does not display all the columns. Therefore use the line below. 
# for col in df_ipeds_temp.columns:
#     print(col)
#     print(col + ": " + str(df_ipeds_original[col].dtype)) # Do this if you want to see the data type. 

In [109]:
# Import the second file. This file contains the graduation rate variable. 

xls_file2 = pd.ExcelFile('CompletionRate.xlsx')
df_grad_ori = xls_file2.parse('Sheet1')
df_grad_ori.dropna(subset=['C150_4'], inplace = True)
df_grad_ori.drop([ 'OPEID','OPEID6','INSTNM'], axis = 1, inplace = True) 
df_grad_ori.rename(index=str, columns={'UNITID': 'unitid'}, inplace =True)
df_grad_ori.rename(index=str, columns={'C150_4': 'grad_rate'}, inplace =True)
# df_grad_ori.head(1)

In [110]:
# In the first file, remove any rows that have null values in the SAT/ACT columns
df_ipeds_temp.dropna(subset=['Calculated SAT M+CR Mean', 'Calculated ACT C Mean', 'Applicants total', 'Admissions total', 
                             'Enrolled total'], inplace = True)

# Create some columns based on calculation

# Normalize the SAT score by dividing 1600 (total possible score)
df_ipeds_temp['Normalized_SAT'] = df_ipeds_temp['Calculated SAT M+CR Mean'] /1600
# Normalize the ACT score by dividing 36 (total possible score)
df_ipeds_temp['Normalized_ACT'] = df_ipeds_temp['Calculated ACT C Mean'] /36
# Get average for the two normalized test score
df_ipeds_temp['Normalized_SAT_ACT']= df_ipeds_temp[['Normalized_SAT','Normalized_ACT']].mean(axis=1)
# Admission_pct, a.k.a. Offer Rate, is to measure the % of applicants who received the offer 
df_ipeds_temp['Admission_pct']= df_ipeds_temp['Admissions total'] / df_ipeds_temp['Applicants total']
# Enroll_pct is to measure the % of offer receiver who eventually enrolled
df_ipeds_temp['Enroll_pct']= df_ipeds_temp['Enrolled total'] / df_ipeds_temp['Admissions total']
# 'Desire' is to measure the ratio of Enroll_pct / Admission_pct
df_ipeds_temp['Desire']= df_ipeds_temp['Enroll_pct'] / df_ipeds_temp['Admission_pct']
# 'Chosen_pct' is to measure the % of who enrolled against applicants. a.k.a. Of total applicants, the % that enrolled. 
df_ipeds_temp['Chosen_pct']= df_ipeds_temp['Enrolled total'] / df_ipeds_temp['Applicants total']
# 'POT_applied' is to measure that of total students(who enrolled in U.S. colleges), the % of them applied a specific college
df_ipeds_temp['POT_applied']= df_ipeds_temp['Applicants total'].div(df_ipeds_temp['Enrolled total'].sum())
# create a new column Tuition_Revised base on Private/Public. This variable is used in some part of the analysis. 
# If 'Public' then Tuition = In State Tuition. Else (will be 'private') then = Out State Tuition
df_ipeds_temp['Tuition_Revised'] = np.where(df_ipeds_temp['Control_of_institution']=='Public', df_ipeds_temp['Tuition_In_State'], df_ipeds_temp['Tuition_Out_State'])
# drop un-necessary columns
df_ipeds_temp.drop([ 'Normalized_SAT','Normalized_ACT'], axis = 1) 

# Left join the data with the second file to include the grad_rate in the same data frame. 
df_ipeds = pd.merge(df_ipeds_temp, df_grad_ori, on=['unitid'], how='left')



The final data frame sample. See below. 

In [111]:
df_ipeds.head(1)
# Alternatively, you can export to csv to see the file easier, since there are too many variables to display on screen. 
# export_csv = df_ipeds_temp.to_csv (r'C:\Test\df_ipeds_temp.csv', index = None, header=True) 

,unitid,Institution name,ZIP,Longitude location of institution,Latitude location of institution,Carnegie Classification 2010: Basic,Religious affiliation,Applicants total,Admissions total,Enrolled total,...,Normalized_SAT,Normalized_ACT,Normalized_SAT_ACT,Admission_pct,Enroll_pct,Desire,Chosen_pct,POT_applied,Tuition_Revised,grad_rate
0,100654,Alabama A & M University,35762,-86.568502,34.783368,Master's Colleges and Universities (larger pro...,Not applicable,6142.0,5521.0,1104.0,...,0.50625,0.472222,0.489236,0.898893,0.199964,0.222456,0.179746,0.004879,5592.0,0.2749


## 2. The Quadrant

If we plot all colleges in a scatter plot, x axis being the offer% (percent of applicants who received the offer), y axis being the enrollment% (percent of offer receivers who eventually enrolled), theoretically, we should see four quadrants (see below picture). 

*Highly Sought After* - The colleges in the upper left corner are those with low offer% and high enroll%. These are the colleges that are highly sought after. They are the top colleges. 

*Insurance* - The college in the lower right corner, are in the exact opposite situation. They gave out a lot of offers, but if the students have better choice, they would choose the better ones instead of these insurance college. 

*Stand the Ground* - The colleges in the lower left corner do not get high enrollment rate, however they stand the ground, do not give out the offer easily. 

*Soul Mate* - These colleges and students are true soul mate to each other. High offer%, high enroll%.  


In [112]:
%%HTML
<div class='tableauPlaceholder' id='viz1550214515903' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;GG&#47;GGZTFP9N9&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;GGZTFP9N9' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;GG&#47;GGZTFP9N9&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550214515903');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

To do the analysis, we could use a 50%-50% threshold to divide the scatter plot into four quadrants. However, the k-mean cluster technique is a better option. Each colleges *must* compete with each other to stand out. If they have many similarities, then they belong to the same group. Hence the cluster technique is better in providing this quadrant. 

In [113]:
# Use K-mean n = 4 to cluster the data. Base on two variables: Enroll_pct, Admission_pct

kmeans_model = skc.KMeans(n_clusters=4)
kmeans_model.fit(df_ipeds[['Enroll_pct', 'Admission_pct']])
centroids = kmeans_model.cluster_centers_
centroids  # these are the centers of the clusters

# Plot the clusters with their centroids.
pd.options.mode.chained_assignment = None  # default='warn'
df_ipeds['cluster_labels_kmeans'] = kmeans_model.labels_
# f = sns.lmplot(y='Enroll_pct', x='Admission_pct', data=df_ipeds, hue='cluster_labels_kmeans',
#                fit_reg=False, height=5, aspect=1.3)
f.ax.scatter(centroids[:,1], centroids[:,0], marker='+', s=100,
             linewidths=2, color='k')  


## 3. Export the File for Tableau Use - Plot the clusters

After the cluster technique was applied (codes were hidden above), the data was exported out as a csv file for Tableau use. 

In [114]:
# Do this ONLY when necessary. K-mean clusters labels change all the time. 
# export_csv = df_ipeds.to_csv (r'C:\Users\LC\605Python\df_ipeds_tableau.csv', index = None, header=True) #export to csv for Tableau

### Question 1: What do the clusters look like? Do they seem all right? 

Below is the result. 

We can see that colleges are grouped into n=4 clusters. While the cluster technique does not allow train/test validation, I am still curious to see if the clusters are 'as expected'. 

Knowing the Ivy League colleges are the top colleges in the U.S., I marked them as stars in the plot below. All of them are in the 'Highly Sought After' group. 

Good! 

In [115]:
%%HTML
<div class='tableauPlaceholder' id='viz1550215454402' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;CB&#47;CBPP62GRZ&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;CBPP62GRZ' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;CB&#47;CBPP62GRZ&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550215454402');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>


## Question 2: What is the ACT/SAT Test Score's Role? 

Many studesnts consider the SAT/ACT test score as the deciding factor for college admission. Therefore, we plot a bar chart for the SAT/ACT score by clusters, as well as the histogram of scores, color by clusters. 

Note: The ACT/SAT score was treated (nomalized) due to their different scoring system. The scores were converted to percentage, rather than the original straight numbers. Then, the average of these two percentage was used as the final ACT/SAT test score percentage. This was done in the initial data cleaning process. 

In [116]:
%%HTML
<div class='tableauPlaceholder' id='viz1550216033300' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;2M&#47;2MMFR2Z8J&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;2MMFR2Z8J' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;2M&#47;2MMFR2Z8J&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550216033300');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Unsurprisingly, the average test score is very high for the Highly Sought After colleges. However, the remaining three clusters' test scores are very close to each other (they are in the same competition zone). 

The purpose of the histogram is not to demonstrate each clusters' proportion in each SAT/ACT test score bin. The purpose is to encourage the students who didn't get a good score. In almost every bin, all four clusters are present. Except for the high-range bins, 80% - 85%, where only two clusters (Highly Sought After and Stand the Ground) present,  then one cluster (Highly Sought After) in 87.5% - 95%. Surprisingly, in the 40% bin, only the Highly Sought After is present. The suspicion is that some students may have special talents which resulted in their admission to the Highly Sought After colleges. But we have no way to confirm this in this data set. 

Conclusion: For the Highly Sought After clusters, generally you do need high test score. However, for all the remaining clusters, the test score does not appear to be the deciding factors. *Test Score somehow matters...depends on which cluster*. 

## Question 3: Does money matter? 

In [117]:
%%HTML
<div class='tableauPlaceholder' id='viz1550217257107' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;G6&#47;G6N3FYDSJ&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;G6N3FYDSJ' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;G6&#47;G6N3FYDSJ&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550217257107');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In this chart, the overwhelming first realization is that the hugh gap of tuition in between the private college and public college, especially between the private college and the public in-state tuition. 

Secondly, the top colleges (Highly Sought After) has the highest tuition, unsurprisingly. A very interesting discovery is that the rank of these clusters don't change whether it's Private, Public out-of-state, or Public in-state. Soul mate colleges have the lowest tuition - this is probably why they are soul mate to the students. 

There was another two charts plotted in Tableau on the Financial Aid aspect. The key take-away is that all four clusters are trying very hard to help the students with financial difficulty. Unfortunately, due to the nature of the data, I was not able to normalize such data to create a side-by-side comparison to determine whether the money matters. Therefore these charts were not presented. 

## Question 4: Using the test score and the tuition data, can we predict the type of college (Public/Private) using K-neighbor and Naive Bays methods? 

In [118]:
# Prepare dataframe for train/test

# Note: Why use the 'Tuition_Out_State' instead of other tuition variables? 
# Because the Tuition_Out_State is a fair measure of the college's attraction to all the out-of-state students. 

ipeds_tuition_score_ori = df_ipeds[['Tuition_Out_State', 'Normalized_SAT_ACT', 'Control_of_institution']].copy()
ipeds_tuition_score_ori
ipeds_tuition_score=ipeds_tuition_score_ori.drop([ 'Control_of_institution'], axis=1)

ipeds_tuition_score_raw = ipeds_tuition_score.columns

In [119]:
# Split data to train/test with Tuition_Out_State and Normalized_SAT_ACT

# ipeds_tuition_score_ori = df_ipeds[['Tuition_Revised', 'Normalized_SAT_ACT', 'Control_of_institution']].copy()
from sklearn.model_selection import train_test_split
X_cols =['Tuition_Out_State','Normalized_SAT_ACT' ]
# X_cols
y=ipeds_tuition_score_ori['Control_of_institution']

f_train, f_test, l_train, l_test = train_test_split(ipeds_tuition_score_ori[ipeds_tuition_score_raw], 
                 ipeds_tuition_score_ori['Control_of_institution'], test_size=0.4, random_state=42)

In [120]:
# Use K-Nearest Method and Plot

n = 2
ipeds_neigh = KNeighborsClassifier(n_neighbors=n) #using k-Nearest method
ipeds_neigh.fit(ipeds_tuition_score_ori[['Tuition_Out_State','Normalized_SAT_ACT']],ipeds_tuition_score_ori['Control_of_institution'])


# Plot the result (K-Nearest)

def plot_scores(df, classifier): 
    xx,yy=np.meshgrid(np.linspace(df['Tuition_Out_State'].min(),df['Tuition_Out_State'].max()+10), 
                      np.linspace(df['Normalized_SAT_ACT'].min(),df['Normalized_SAT_ACT'].max()+0.1), 
                     )  
    #df['Normalized SAT'].min(),df['Normalized SAT'].max()+4
    Zstring = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    repdict = {'Public' : 1, 'Private not-for-profit' : 2
              } #map predictions to plottable numbers

    Z = np.zeros_like(Zstring)
    for k,v in repdict.items():
        Z[Zstring==k]=v
    
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.get_cmap('RdBu',6),vmin=0,vmax=5)
    plt.xlabel('Tuition_Out_State');plt.ylabel('Normalized_SAT_ACT')
    cbar = plt.colorbar(ticks=[1,2])
    cbar.ax.set_yticklabels(['Public', 'Private'])
    plt.scatter(df['Tuition_Out_State'],df['Normalized_SAT_ACT'], color="black",alpha=0.2) 
    
# plot_scores(f_test,ipeds_neigh)
# plt.title('K-Nearest Neighbors', fontsize = 16)

In [121]:
# Use Naive Bays Method and Plot

# from sklearn.naive_bayes import GaussianNB  

# ipeds_gnb = GaussianNB().fit(f_train,l_train)
# ipeds_gnb_predictions_test = ipeds_gnb.predict(f_test)

# # Plot the result (Naive Bays)
# plot_scores(ipeds_tuition_score,ipeds_gnb)
# plt.title('Naive Bayes', fontsize = 16)

In [122]:
# This is just to show the actual data - Scatter plot. Color by Public/Private
# sns.lmplot(y='Normalized_SAT_ACT', x='Tuition_Out_State', hue='Control_of_institution', 
#            palette = {'Public':'tomato', 'Private not-for-profit':'peachpuff'}, legend = False,
#            fit_reg=False, data=df_ipeds);
# plt.title('Normalized_SAT_ACT vs. Tuition_Out_State')

In [123]:
%%HTML
<div class='tableauPlaceholder' id='viz1550218403222' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;D6&#47;D6CTRJ9JF&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;D6CTRJ9JF' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;D6&#47;D6CTRJ9JF&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550218403222');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

* Note: Originally, I used the variable "Tuition_Revised" for the tuition. "Tuition_Revised" is a calculated varilable that when a college is public, then use the In-State Tuition, else Out-Stat Tuition. Later, I realized that it was not a fair comparison - The public colleges accept out-of-state students too. Therefore, since the out-of-state tuition is the same for either private or public, I decided to use the out-of-state tuition as the variable in this analysis. 

It is very surprising to see that both methods predict the type of colleges solely by tuition. This, on the other hand, somehow corresponds to the conclusion in Question 2 - the test score doesn't seem to be significant in the three clusters (Stand the Ground, Soul Mate and Insurance). But tuition matters. 

A side question was raised - Since the tuition is so dominating, do the private colleges accept students regardless of their test score, then subsequently causes low graduation rate? 

Base on the average graduation rate chart, the answer is 'no'. On the contrary, the private colleges actually deliver higher graduation rate. 

## Question 5: Any Regression Model? 

In this exercise, I chose a variety of variables to build the model. The combination includes: 

Different y variables: 1. Desire index (% Enrolled / % Offer); 2.  % applicants who enrolled (# of Enrolled / # of Applicant), different from the enrollment rate. 3. % of total later-college-students who applied (# of Applicant / # all available seats in U.S. POT_applied)

Different x variables: Normalized_ACT_SAT, Tuition_Out_State, Tuition_Revised (If public college, Tuition_In_State, else Tuition_Out_State), grad_rate, cluster

Different terms: First order model only vs. model with interaction terms 

Different Data Frame: Private college only, Public college only, All college


In [124]:
# Regression 1:  
# Y: Desire.
# X: Normalized_SAT_ACT, Tuition_Out_State OR Tuition_Revised, grad_rate
# Data Frame: varies. Public only / Private only / Combined

df_ipeds.dropna(subset=['grad_rate'], inplace = True)

import statsmodels.formula.api as smf

df_ipeds_private=df_ipeds[(df_ipeds.Control_of_institution == "Private not-for-profit") ]
df_ipeds_public=df_ipeds[(df_ipeds.Control_of_institution == "Public") ]

Private_interac_desire_g = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate+ Normalized_SAT_ACT*Tuition_Out_State+ Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds_private).fit() 
Private_desire_g = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate', data=df_ipeds_private).fit() 
Public_interac_desire_g = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate+ Normalized_SAT_ACT*Tuition_Out_State+ Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds_public).fit() 
Public_desire_g = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate', data=df_ipeds_public).fit() 
Combine_interac_desire_g = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate+ Normalized_SAT_ACT*Tuition_Revised+ Tuition_Revised*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds).fit() 
Combine_desire_g = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate', data=df_ipeds).fit() 


df_R2adj_desire_g = pd.DataFrame({'Model': [ 'Private_interac_desire_g', 'Private_desire_g', 'Public_interac_desire_g', 'Public_desire_g', 'Combine_interac_desire_g', 'Combine_desire_g'], 
                   'R2_Adjusted': [Private_interac_desire_g.rsquared_adj,Private_desire_g.rsquared_adj, Public_interac_desire_g.rsquared_adj, Public_desire_g.rsquared_adj, Combine_interac_desire_g.rsquared_adj, Combine_desire_g.rsquared_adj]})
# df_R2adj_desire_g = pd.DataFrame({'Model': [ 'Private_desire_g',  'Public_desire_g',  'Combine_desire_g'], 
#                    'R2_Adjusted': [Private_desire_g.rsquared_adj, Public_desire_g.rsquared_adj, Combine_desire_g.rsquared_adj]})
# df_R2adj_desire_g


In [125]:
# Regression 2:  
# Y: Chosen_pct.
# X: Normalized_SAT_ACT, Tuition_Out_State OR Tuition_Revised, grad_rate
# Data Frame: varies. Public only / Private only / Combined

df_ipeds.dropna(subset=['grad_rate'], inplace = True)

import statsmodels.formula.api as smf

df_ipeds_private=df_ipeds[(df_ipeds.Control_of_institution == "Private not-for-profit") ]
df_ipeds_public=df_ipeds[(df_ipeds.Control_of_institution == "Public") ]

Private_interac_chosen_g = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + Normalized_SAT_ACT*Tuition_Out_State + Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds_private).fit() 
Private_chosen_g = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate', data=df_ipeds_private).fit() 
Public_interac_chosen_g = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + Normalized_SAT_ACT*Tuition_Out_State + Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds_public).fit() 
Public_chosen_g = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate', data=df_ipeds_public).fit() 
Combine_interac_chosen_g = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate + Normalized_SAT_ACT*Tuition_Revised + Tuition_Revised*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds).fit() 
Combine_chosen_g = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate', data=df_ipeds).fit() 

df_R2adj_chosen_g = pd.DataFrame({'Model': ['Private_interac_chosen_g', 'Private_chosen_g', 'Public_interac_chosen_g', 
                                          'Public_chosen_g', 'Combine_interac_chosen_g', 'Combine_chosen_g'], 
                   'R2_Adjusted': [Private_interac_chosen_g.rsquared_adj,Private_chosen_g.rsquared_adj, 
                              Public_interac_chosen_g.rsquared_adj, Public_chosen_g.rsquared_adj, 
                              Combine_interac_chosen_g.rsquared_adj, Combine_chosen_g.rsquared_adj]})
# df_R2adj_chosen_g

In [126]:
# Regression 3:  
# Y: POT_applied.
# X: Normalized_SAT_ACT, Tuition_Out_State OR Tuition_Revised, grad_rate
# Data Frame: varies. Public only / Private only / Combined

df_ipeds.dropna(subset=['grad_rate'], inplace = True)
import statsmodels.formula.api as smf

df_ipeds_private=df_ipeds[(df_ipeds.Control_of_institution == "Private not-for-profit") ]
df_ipeds_public=df_ipeds[(df_ipeds.Control_of_institution == "Public") ]

Private_interac_POT_applied_g = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + Normalized_SAT_ACT*Tuition_Out_State + Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds_private).fit() 
Private_POT_applied_g = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate', data=df_ipeds_private).fit() 
Public_interac_POT_applied_g = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + Normalized_SAT_ACT*Tuition_Out_State + Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds_public).fit() 
Public_POT_applied_g = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate', data=df_ipeds_public).fit() 
Combine_POT_interac_applied_g = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate + Normalized_SAT_ACT*Tuition_Revised + Tuition_Revised*grad_rate + Normalized_SAT_ACT*grad_rate', data=df_ipeds).fit() 
Combine_POT_applied_g = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate', data=df_ipeds).fit() 

df_R2adj_POT_applied_g = pd.DataFrame({'Model': ['Private_interac_POT_applied_g', 'Private_POT_applied_g', 'Public_interac_POT_applied_g', 
                                          'Public_POT_applied_g', 'Combine_POT_interac_applied_g', 'Combine_POT_applied_g',], 
                   'R2_Adjusted': [Private_interac_POT_applied_g.rsquared_adj,Private_POT_applied_g.rsquared_adj, 
                              Public_interac_POT_applied_g.rsquared_adj, Public_POT_applied_g.rsquared_adj, 
                              Combine_POT_interac_applied_g.rsquared_adj, Combine_POT_applied_g.rsquared_adj]})
# df_R2adj_POT_applied_g

In [127]:
# Regression 4:  
# Y: Desire.
# X: Normalized_SAT_ACT, Tuition_Out_State OR Tuition_Revised, grad_rate, cluster
# Data Frame: varies. Public only / Private only / Combined

df_ipeds.dropna(subset=['grad_rate'], inplace = True)

import statsmodels.formula.api as smf

df_ipeds_private=df_ipeds[(df_ipeds.Control_of_institution == "Private not-for-profit") ]
df_ipeds_public=df_ipeds[(df_ipeds.Control_of_institution == "Public") ]

Private_interac_desire_gc = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate+ Normalized_SAT_ACT*Tuition_Out_State+ Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate + cluster_labels_kmeans', data=df_ipeds_private).fit() 
Private_desire_gc = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + cluster_labels_kmeans', data=df_ipeds_private).fit() 
Public_interac_desire_gc = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate+ Normalized_SAT_ACT*Tuition_Out_State+ Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate + cluster_labels_kmeans', data=df_ipeds_public).fit() 
Public_desire_gc = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + cluster_labels_kmeans', data=df_ipeds_public).fit() 
Combine_interac_desire_gc = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate+ Normalized_SAT_ACT*Tuition_Revised+ Tuition_Revised*grad_rate + Normalized_SAT_ACT*grad_rate + cluster_labels_kmeans', data=df_ipeds).fit() 
Combine_desire_gc = smf.ols(formula='Desire ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate + cluster_labels_kmeans', data=df_ipeds).fit() 


df_R2adj_desire_gc = pd.DataFrame({'Model': [ 'Private_interac_desire_gc', 'Private_desire_gc', 'Public_interac_desire_gc', 'Public_desire_gc', 'Combine_interac_desire_gc', 'Combine_desire_gc'], 
                   'R2_Adjusted': [Private_interac_desire_gc.rsquared_adj,Private_desire_gc.rsquared_adj, Public_interac_desire_gc.rsquared_adj, Public_desire_gc.rsquared_adj, Combine_interac_desire_gc.rsquared_adj, Combine_desire_gc.rsquared_adj]})
# df_R2adj_desire_g = pd.DataFrame({'Model': [ 'Private_desire_g',  'Public_desire_g',  'Combine_desire_g'], 
#                    'R2_Adjusted': [Private_desire_g.rsquared_adj, Public_desire_g.rsquared_adj, Combine_desire_g.rsquared_adj]})
# df_R2adj_desire_gc


In [128]:
# Regression 5:  
# Y: Chosen_pct.
# X: Normalized_SAT_ACT, Tuition_Out_State OR Tuition_Revised, grad_rate, cluster
# Data Frame: varies. Public only / Private only / Combined

df_ipeds.dropna(subset=['grad_rate'], inplace = True)

import statsmodels.formula.api as smf

df_ipeds_private=df_ipeds[(df_ipeds.Control_of_institution == "Private not-for-profit") ]
df_ipeds_public=df_ipeds[(df_ipeds.Control_of_institution == "Public") ]

Private_interac_chosen_gc = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + Normalized_SAT_ACT*Tuition_Out_State + Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate + C(cluster_labels_kmeans)', data=df_ipeds_private).fit() 
Private_chosen_gc = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + C(cluster_labels_kmeans)', data=df_ipeds_private).fit() 
Public_interac_chosen_gc = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + Normalized_SAT_ACT*Tuition_Out_State + Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate + C(cluster_labels_kmeans)', data=df_ipeds_public).fit() 
Public_chosen_gc = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate+ C(cluster_labels_kmeans)', data=df_ipeds_public).fit() 
Combine_interac_chosen_gc = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate + Normalized_SAT_ACT*Tuition_Revised + Tuition_Revised*grad_rate + Normalized_SAT_ACT*grad_rate+ C(cluster_labels_kmeans)', data=df_ipeds).fit() 
Combine_chosen_gc = smf.ols(formula='Chosen_pct ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate+ C(cluster_labels_kmeans)', data=df_ipeds).fit() 

df_R2adj_chosen_gc = pd.DataFrame({'Model': ['Private_interac_chosen_gc', 'Private_chosen_gc', 'Public_interac_chosen_gc', 
                                          'Public_chosen_gc', 'Combine_interac_chosen_gc', 'Combine_chosen_gc'], 
                   'R2_Adjusted': [Private_interac_chosen_gc.rsquared_adj,Private_chosen_gc.rsquared_adj, 
                              Public_interac_chosen_gc.rsquared_adj, Public_chosen_gc.rsquared_adj, 
                              Combine_interac_chosen_gc.rsquared_adj, Combine_chosen_gc.rsquared_adj]})
# df_R2adj_chosen_gc
# Private_interac_chosen_gc.summary()

In [129]:
# Regression 6:  
# Y: POT_Applied.
# X: Normalized_SAT_ACT, Tuition_Out_State OR Tuition_Revised, grad_rate, cluster
# Data Frame: varies. Public only / Private only / Combined

df_ipeds.dropna(subset=['grad_rate'], inplace = True)
import statsmodels.formula.api as smf

df_ipeds_private=df_ipeds[(df_ipeds.Control_of_institution == "Private not-for-profit") ]
df_ipeds_public=df_ipeds[(df_ipeds.Control_of_institution == "Public") ]

Private_interac_POT_applied_gc = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + Normalized_SAT_ACT*Tuition_Out_State + Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate + C(cluster_labels_kmeans)', data=df_ipeds_private).fit() 
Private_POT_applied_gc = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate+ C(cluster_labels_kmeans)', data=df_ipeds_private).fit() 
Public_interac_POT_applied_gc = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate + Normalized_SAT_ACT*Tuition_Out_State + Tuition_Out_State*grad_rate + Normalized_SAT_ACT*grad_rate+ C(cluster_labels_kmeans)', data=df_ipeds_public).fit() 
Public_POT_applied_gc = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Out_State + grad_rate+ C(cluster_labels_kmeans)', data=df_ipeds_public).fit() 
Combine_POT_interac_applied_gc = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate + Normalized_SAT_ACT*Tuition_Revised + Tuition_Revised*grad_rate + Normalized_SAT_ACT*grad_rate+ C(cluster_labels_kmeans)', data=df_ipeds).fit() 
Combine_POT_applied_gc = smf.ols(formula='POT_applied ~ Normalized_SAT_ACT + Tuition_Revised + grad_rate+ C(cluster_labels_kmeans)', data=df_ipeds).fit() 

df_R2adj_POT_applied_gc = pd.DataFrame({'Model': ['Private_interac_POT_applied_gc', 'Private_POT_applied_gc', 'Public_interac_POT_applied_gc', 
                                          'Public_POT_applied_gc', 'Combine_POT_interac_applied_gc', 'Combine_POT_applied_gc',], 
                   'R2_Adjusted': [Private_interac_POT_applied_gc.rsquared_adj,Private_POT_applied_gc.rsquared_adj, 
                              Public_interac_POT_applied_gc.rsquared_adj, Public_POT_applied_gc.rsquared_adj, 
                              Combine_POT_interac_applied_gc.rsquared_adj, Combine_POT_applied_gc.rsquared_adj]})
# df_R2adj_POT_applied_gc

In [130]:
%%HTML
<div class='tableauPlaceholder' id='viz1550219662426' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;G2&#47;G2WW298WC&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;G2WW298WC' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;G2&#47;G2WW298WC&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550219662426');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>


*Limitation: The flaw of this data set was demonstrated in this analysis. The application/admission/enrollment data was not separated by in-state/out-of-state. 

None of the regression models deliver a high r2_adj values (highest 0.64). This indicates that more variables need to be explored. 

### I really feel the need to demonstrate this result to Chinese parents. Since thousands of years ago in China (and currently), the test score has ALWAYS been the SOLE deciding factor for college entrance.  These Chinese parents feel that they are being discriminated in the U.S. in the college entrance issue due to their race - maybe that's not the case. It's only due to culture difference and lack-of-communication. 



## Hidden Question: Mike's Question

I choose not to present this question in the data story due to the time restriction. But I really want to do this analysis. There are always people who are lower than average. After all, the society is made up of normal distribution. If somebody is better than average, then there got to be somebody worse than average. Sometimes all those below-average personnel just need a chance - a chance to help them to move up. 

In [131]:
%%HTML
<div class='tableauPlaceholder' id='viz1550220608608' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;45&#47;45TFFJF4M&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;45TFFJF4M' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;45&#47;45TFFJF4M&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1550220608608');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Above are a few colleges who fit the need for Mike. I hope students like Mike can take advantage of this analysis and move on well later in their life. 

# Conclusion

#### Test score, tuition, cluster of colleges all play some roles in the college application/admission process. For top colleges, test score and tuition are very obvious factors. For the other three clusters, these variables' impacts are not that significant. 

#### It is also obvious that the above mentioned variables are not the sole deciding factors. Additional variables are required to include in this study to generate a final model. 

## Limitation

This data set does not separate the admission/enrollment data by in-state/out-of-state, which is a very important factor. In-State student pays much lower tuition than out-of-state students in public school.  The regression model fit should be improved should such data become available. 

This data set does not contain any college ranking data. If this variable is available, there could be many new interesting analysis in each of the slides. 



# Thank you